In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import time
import datetime
from datetime import datetime
import calendar

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

sns.set_style('white')


pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
df_sorted = pd.read_pickle('/kaggle/input/easymoney/EasyMoney_base.pkl',compression='zip')

In [ ]:
df_sorted .info()

Hacemos una lista con las variables dif_ que vamos a usar:

In [ ]:
lst_dif=['dif_debit_card',              
'dif_em_account_p',               
'dif_em_account_pp',              
'dif_em_acount',                  
'dif_emc_account',                
'dif_payroll',                   
'dif_payroll_account',            
'dif_funds',                      
'dif_long_term_deposit',         
'dif_mortgage',                   
'dif_pension_plan',               
'dif_securities',                
'dif_short_term_deposit',        
'dif_loans',                      
'dif_credit_card']   

Hacemos un melt del dataset para las variables anteriores.

In [ ]:
df_=df_sorted.melt(id_vars=['pk_partition','pk_cid'],
              value_vars=lst_dif,
              var_name='Product',
              value_name='Count')

In [ ]:
df_

In [ ]:
len(df_)

Vemos que casi todos los valores son 0. hay 334000 altas y 197000 bajas

In [ ]:
df_['Count'].value_counts()

Vemos para un cliente en concreto:

In [ ]:
df_[df_['pk_cid']==17457]['Count'].value_counts()

Nos creamos un nuevo dataset solo con las altas:

In [ ]:
df=df_[df_['Count']==1]

In [ ]:
df.head()

Hacemos un diccionario con los productos y lo que se ingresa por ellos

In [ ]:
ingresos={'dif_debit_card':10 ,              
'dif_em_account_p':10 ,               
'dif_em_account_pp':10 ,              
'dif_em_acount':10 ,                  
'dif_emc_account':10 ,                
'dif_payroll':10 ,                   
'dif_payroll_account':10 ,            
'dif_funds':40 ,                      
'dif_long_term_deposit':40 ,         
'dif_mortgage':40 ,                   
'dif_pension_plan':40 ,               
'dif_securities':40 ,                
'dif_short_term_deposit':40 ,        
'dif_loans':60 ,                      
'dif_credit_card':60 }

Se lo mapeamos al dataset usando el campo Product

In [ ]:
df['ingresosProducto']=df['Product'].map(ingresos)

Vemos como queda:

In [ ]:
df.head()

Multiplicamos las altas por lo que se ingresa:

In [ ]:
df['ingresos']=df['Count']*df['ingresosProducto']

Ya lo tenemos:

In [ ]:
df.head()

Agrupamos por mes y cliente para que se sumen los ingresos de todas las altas de varios productos si las hay.

In [ ]:
clientes_ingresos=df.groupby(['pk_partition','pk_cid']).agg(
    {'ingresosProducto':np.sum}).reset_index(drop=False)

Vemos que clientes nos han generado mas beneficio en un mes:

In [ ]:
clientes_ingresos.sort_values(by='ingresosProducto',ascending=False)

Vamos a unirselo al dataset principal, pero vemos que va a haber muchos nulos. 

In [ ]:
len(df_sorted),len(clientes_ingresos)

In [ ]:
df_sorted =pd.merge(df_sorted , clientes_ingresos, 
                    how="left",on=['pk_cid','pk_partition'])

In [ ]:
df_sorted.isnull().sum()

Rellenamos los nulos con 0, ya que estos representan las bajas y los meses que no hay cambios.0 €

In [ ]:
df_sorted['ingresosProducto'].fillna(0,inplace=True)

In [ ]:
df_sorted.isnull().sum().sum()

Explolramos los datos y vemos que la informacion es consistente y coincide.

In [ ]:
df_sorted[df_sorted['ingresosProducto']>10].T

In [ ]:
df_sorted[df_sorted['ingresosProducto']==0].head()

In [ ]:
df_sorted[df_sorted['pk_cid']==17457].T

In [ ]:
clientes_ingresos

In [ ]:
clientes_ingresos.groupby('pk_cid')['ingresosProducto'].sum().sort_values(ascending=False)

In [ ]:
df_sorted['maximoBeneficio']=df_sorted['pk_cid'].map(clientes_ingresos.groupby('pk_cid')['ingresosProducto'].sum().sort_values(ascending=False))

In [ ]:
df_sorted[df_sorted['pk_cid']==17457].T